# Otimização de Portfólio
Otimização de portfólio usando a Fronteira Eficiente.

In [ ]:
import pandas as pd
import numpy as np
import datetime
import pandas_datareader as pdr
import plotly
import plotly.offline as plo
import plotly.graph_objs as plg
import plotly.express as plx
from scipy.stats import norm

### Funções de Configurações Gráficas
Funções para configurações base de gráficos e exibição.

In [ ]:
def Plot_Formato(fig, title):
    title={'text': title, 'xanchor': 'center', 'yanchor': 'bottom', 'y':0, 'x':0.5,}
    
    fig.update_layout(title=title, 
            xaxis_rangeslider_visible=False,  width=1280, height=720,
            xaxis_showgrid=True, xaxis_gridwidth=1, xaxis_gridcolor='#E8E8E8',
            yaxis_showgrid=True, yaxis_gridwidth=1, yaxis_gridcolor='#E8E8E8',
            plot_bgcolor='rgba(0,0,0,0)')
    
def Plot_Show(data, title):
    fig = plg.Figure(data=data)

    Plot_Formato(fig, title)

    fig.show()   

### Leitura de Dados de Ativos
Realiza a leitura online de dados de ativos financeiros.

In [ ]:
#Acoes = ["ITUB4.SA", "CPLE3.SA", "LREN3.SA", "EMBR3.SA", "PSSA3.SA", "GOAU4.SA"]
Acoes = ["AAPL", "NKE", "GOOGL", "AMZN"]
Periodo_Inicio = datetime.datetime(2015,1,1)
Periodo_Termino = datetime.datetime(2019,12,31)

In [ ]:
DadosAcoes = pdr.get_data_yahoo(Acoes, start=Periodo_Inicio, end=Periodo_Termino)

In [ ]:
Dados = DadosAcoes["Adj Close"]

In [ ]:
Dados

### Fronteira Eficiente
Calcula o retorno esperado para um portfólio e cria fronteira eficiente simulando 10.000 portfólios.

- Retorno dos Ativos

In [ ]:
DadosRetorno = Dados.pct_change().fillna(0)

In [ ]:
DadosRetorno

- Matriz de Covariância

In [ ]:
DadosCovariancia = DadosRetorno.cov()

In [ ]:
DadosCovariancia

- Matriz de Correlação

In [ ]:
DadosCorrelacao = DadosRetorno.corr()

In [ ]:
DadosCorrelacao

- Definição do Portfolio e Variância

In [ ]:
Pesos = {'AAPL': 0.1, 'NKE': 0.2, 'GOOGL': 0.5, 'AMZN': 0.2}

In [ ]:
PortfolioVariancia = DadosCovariancia.mul(Pesos, axis=0).mul(Pesos, axis=1).sum().sum()

In [ ]:
PortfolioVariancia

- Retorno Esperado do Portfólio

In [ ]:
RetornoAnualizado = Dados.resample("Y").last().pct_change().mean()

In [ ]:
RetornoAnualizado

In [ ]:
PortfolioRetorno = (list(Pesos.values()) * RetornoAnualizado).sum()

In [ ]:
PortfolioRetorno

- Simulação de Portfólios

In [ ]:
RetornosSimulados = []
RiscosSimulados = []
PesosSimulados = []

NumAtivos = len(Dados.columns)
NumPortfolios = 10000

In [ ]:
for portfolio in range(NumPortfolios):
    pesos = np.random.random(NumAtivos)
    pesos = pesos/np.sum(pesos)
    PesosSimulados.append(pesos)
    retornos = np.dot(pesos, RetornoAnualizado)
 
    RetornosSimulados.append(retornos)
    var = DadosCovariancia.mul(pesos, axis=0).mul(pesos, axis=1).sum().sum()
    sd = np.sqrt(var)
    ann_sd = sd*np.sqrt(250)
    RiscosSimulados.append(ann_sd)

In [ ]:
data = {'Retorno':RetornosSimulados, 'Risco':RiscosSimulados}
for counter, symbol in enumerate(Dados.columns.tolist()):
    data[symbol] = [w[counter] for w in PesosSimulados]

In [ ]:
Portfolios = pd.DataFrame(data)
Portfolios.head()

- Performance de cada Portfólio

In [ ]:
Portfolios["Sharpe"] = Portfolios["Retorno"] / Portfolios["Risco"]

In [ ]:
Portfolios

- Fronteira Eficiente Visual

In [ ]:
FronteiraEficiente = plg.Figure(plg.Scatter(x=Portfolios["Risco"],
                                            y=Portfolios["Retorno"],
                                            mode="markers", marker_symbol="circle", marker_opacity=0.5,
                                            name="Portfolios Simulados"))
Plot_Show(FronteiraEficiente, "Fronteira Eficiente")

- Portfólio de Menor Risco

In [ ]:
PortfolioMenorRisco = Portfolios.iloc[Portfolios["Risco"].idxmin()]

In [ ]:
PortfolioMenorRisco

- Portfólio de Maior Retorno

In [ ]:
PortfolioMaiorRetorno = Portfolios.iloc[Portfolios["Risco"].idxmax()]

In [ ]:
PortfolioMaiorRetorno

- Portfólio de Melhor Performance

In [ ]:
PortfolioMelhorSharpe = Portfolios.iloc[Portfolios["Sharpe"].idxmax()]

In [ ]:
PortfolioMelhorSharpe

In [ ]:
FronteiraEficiente = plg.Figure()

FronteiraEficiente.add_trace(plg.Scatter(x=Portfolios["Risco"],
                                         y=Portfolios["Retorno"],
                                         mode="markers", marker_symbol="circle", marker_opacity=0.5,
                                         name="Portfolios Simulados"))
FronteiraEficiente.add_trace(plg.Scatter(x=[PortfolioMenorRisco[1]],
                                         y=[PortfolioMenorRisco[0]],
                                         mode="markers", marker_symbol="star", marker_size=20,
                                         name="Menor Risco"))
FronteiraEficiente.add_trace(plg.Scatter(x=[PortfolioMaiorRetorno[1]],
                                         y=[PortfolioMaiorRetorno[0]],
                                         mode="markers", marker_symbol="star", marker_size=20,
                                         name="Maior Retorno"))
FronteiraEficiente.add_trace(plg.Scatter(x=[PortfolioMelhorSharpe[1]],
                                         y=[PortfolioMelhorSharpe[0]],
                                         mode="markers", marker_symbol="star", marker_size=20,
                                         name="Maior Sharpe"))

Plot_Show(FronteiraEficiente, "Fronteira Eficiente")